In [1]:
import tqdm
import numpy as np
import pandas as pd
import torch
import random
from collections import defaultdict
from transformers import BertTokenizer, EncoderDecoderModel, AutoModelForSequenceClassification

from readers import tg_reader, lenta_reader, ria_reader
from custom_datasets.agency_title_dataset import AgencyTitleDatasetGeneration

In [2]:
ls /home/aobuhtijarov/models/

agency_discr_trained_on_cluster_dataset/  pretrained_dec_6_layers/
agency_discriminator/                     pretrained_enc_8_layers/
agency_discriminator_tuned/               rubert_cased_L-12_H-768_A-12_pt/
gen_title_lenta_ria/                      style_gen_title_from_pretrained/
lenta_ria_clustering_model/               tg_gen_title_1500/
lenta_ria_cross_style/                    tg_gen_title_bottleneck/
paired_clf_from_rubert/


In [3]:
tokenizer_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/'

direct_model_path = '/home/aobuhtijarov/models/style_gen_title_from_pretrained/checkpoint-6000/'
cross_model_path = '/home/aobuhtijarov/models/lenta_ria_cross_style/checkpoint-6000/'
discr_model_path = '/home/aobuhtijarov/models/agency_discriminator_tuned/checkpoint-4000/'

lenta_path = '/home/aobuhtijarov/datasets/lenta/lenta-ru-news.test.csv'
ria_path = '/home/aobuhtijarov/datasets/ria/ria.shuffled.test.json'

In [4]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=False, do_basic_tokenize=False)

In [6]:
direct_model = EncoderDecoderModel.from_pretrained(direct_model_path)
discriminator = AutoModelForSequenceClassification.from_pretrained(discr_model_path)

In [5]:
import json

def reader(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line.strip())

In [6]:
records = [r for r in tqdm.tqdm(reader('../../datasets/full_lenta_ria.test.jsonl'))]

lenta_records = [
    {'text': r['lenta_text'], 'title': r['lenta_title'], 'agency': 'lenta.ru', 'date': r['lenta_date']} 
    for r in records
]

ria_records = [
    {'text': r['ria_text'], 'title': r['ria_title'], 'agency': 'РИА Новости', 'date': r['lenta_date']} 
    for r in records
]

2000it [00:00, 2201.12it/s]


In [7]:
agency_list = ["РИА Новости", "lenta.ru"]
agency_to_special_token_id = {a: tokenizer.vocab[f'[unused{i+1}]'] for i, a in enumerate(agency_list)}
agency_to_discr_target = {a: i for i, a in enumerate(sorted(agency_list))}

In [10]:
ria_data = AgencyTitleDatasetGeneration(ria_records, tokenizer, filter_agencies=None,
                                        agency_to_special_token_id=agency_to_special_token_id)

lenta_data = AgencyTitleDatasetGeneration(lenta_records, tokenizer, filter_agencies=None, 
                                          agency_to_special_token_id=agency_to_special_token_id)

In [11]:
len(ria_data), len(lenta_data)

(2000, 2000)

In [12]:
@torch.no_grad()
def eval_direct_on_dataset(dataset, target_agency, n=1000, max_tokens_title=48):
    y_pred = []


    for i in tqdm.tqdm(np.random.choice(len(dataset), n, replace=False), total=n):
        x = dataset[i]
        x['input_ids'][1] = agency_to_special_token_id[target_agency]

        gen_ids = direct_model.generate(
            input_ids=x['input_ids'].cuda().unsqueeze(0),
            attention_mask=x['attention_mask'].cuda().unsqueeze(0),
            decoder_start_token_id=direct_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )
        
        gen_title = [tokenizer.decode(x, skip_special_tokens=True) for x in gen_ids][0]
        
        inp = tokenizer(gen_title, 
            add_special_tokens=True, max_length=max_tokens_title,
            padding='max_length', truncation=True
        )

        logits = discriminator(input_ids=torch.LongTensor(inp['input_ids']).cuda().unsqueeze(0), 
                               attention_mask=torch.LongTensor(inp['attention_mask']).cuda().unsqueeze(0))[0]
        y_pred.append(torch.argmax(logits).item())
    
    return y_pred.count(agency_to_discr_target[target_agency]) / n


In [11]:
def gen_title(input_ids, attention_mask):
    gen_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_start_token_id=model.config.decoder.pad_token_id,
        min_length=7,
        max_length=22,
        num_beams=6
    )

    gen_title = [tokenizer.decode(x, skip_special_tokens=True) for x in gen_ids][0]
    return gen_title

In [ ]:
max_tokens_title = 48

In [ ]:
agency_to_discr_target

In [14]:
n = random.randint(0, 43000)
n = 11306
# n=1186
x = lenta_data[n]
a = gen_title(x['input_ids'].cuda().unsqueeze(0), x['attention_mask'].cuda().unsqueeze(0))
b = lenta_records[n]['title']
print('Gen:', a)
print('Ref:', b)

inp = tokenizer(a, 
    add_special_tokens=True, max_length=max_tokens_title,
    padding='max_length', truncation=True
)

logits = discriminator(input_ids=torch.LongTensor(inp['input_ids']).cuda().unsqueeze(0), 
                       attention_mask=torch.LongTensor(inp['attention_mask']).cuda().unsqueeze(0))[0]
a_pred = torch.argmax(logits).item()

inp = tokenizer(b, 
    add_special_tokens=True, max_length=max_tokens_title,
    padding='max_length', truncation=True
)

logits = discriminator(input_ids=torch.LongTensor(inp['input_ids']).cuda().unsqueeze(0), 
                       attention_mask=torch.LongTensor(inp['attention_mask']).cuda().unsqueeze(0))[0]
b_pred = torch.argmax(logits).item()

print('Gen:', a_pred)
print('Ref:', b_pred)

tensor([[    0, 20622, 42873, 96388,  6188, 45206, 34124, 16327, 12938,  1650,
         27453,   131, 17407, 29079, 17609, 24461,   102,   102,   102,   102,
           102,   102]], device='cuda:0')
Gen: адвокат емельяненко допустил оспаривание решения об условно-досрочном освобождении
Ref: адвокат потерпевшей рассказал о возможном оспаривании удо александра емельяненко
Gen: 0
Ref: 0


In [ ]:
agency_list

In [12]:
%%time

direct_ria_acc = eval_on_dataset(ria_data, agency_list[0])
direct_lenta_acc = eval_on_dataset(lenta_data, agency_list[1])

100%|██████████| 1000/1000 [06:27<00:00,  2.58it/s]

CPU times: user 37min 58s, sys: 8.07 s, total: 38min 6s
Wall time: 37min 57s


## Full prepend eval

In [21]:
direct_model.cuda();
discriminator.cuda();

In [22]:
%%time

result = defaultdict(list)

for i, data in enumerate((ria_data, lenta_data)):
    for target_a in agency_list:
        acc = eval_direct_on_dataset(data, target_a)
        result[i].append(acc)

100%|██████████| 1000/1000 [04:14<00:00,  3.93it/s]

CPU times: user 18min 25s, sys: 2.36 s, total: 18min 27s
Wall time: 18min 25s


In [23]:
df = pd.DataFrame(columns=['Data'] + agency_list)

for i, dataset_name in enumerate(('RIA', 'Lenta')):
    row = {'Data': dataset_name}
    for j, a in enumerate(agency_list):
        row[a] = result[i][j]      
    df = df.append(row, ignore_index=True)
df

,Data,РИА Новости,lenta.ru
0,RIA,0.904,0.109
1,Lenta,0.545,0.430


In [24]:
df.index = df.Data

df.drop('Data', axis=1, inplace=True)

In [26]:
df.round(2)

,РИА Новости,lenta.ru
Data,,
RIA,0.90,0.11
Lenta,0.55,0.43


# Cross style check woth paired classifier

In [8]:
from custom_datasets import LentaRiaDataset

In [9]:
paired_discr_path = '/home/aobuhtijarov/models/paired_clf_from_rubert/'

In [10]:
cross_model = EncoderDecoderModel.from_pretrained(cross_model_path)
cross_model.cuda();

In [11]:
dataset = LentaRiaDataset('/home/aobuhtijarov/datasets/full_lenta_ria.test.jsonl',
                          tokenizer, 250, 48)

In [12]:
paired_discriminator = AutoModelForSequenceClassification.from_pretrained(paired_discr_path)
paired_discriminator.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [13]:
def first_sent(x, token_id):
    lx = list(x)
    if token_id in x:
        return x[:lx.index(token_id)]
    return x

In [14]:
paired_discriminator.eval()
cross_model.eval();

In [15]:
device='cpu'
device='cuda'

In [ ]:
with torch.no_grad():
    p0_vals = []
    p1_vals = []
    
    ria_style_headlines = []
    lenta_style_headlines = []

    for i in tqdm.trange(0, len(dataset), 2):
        x_ria_zag = dataset[i]
        x_lenta_zag = dataset[i+1]
        
        gen_ids_ria_zag = cross_model.generate(
            input_ids=x_ria_zag['input_ids'].to(device).unsqueeze(0),
            attention_mask=x_ria_zag['attention_mask'].to(device).unsqueeze(0),
            decoder_start_token_id=cross_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )
        
        gen_ids_lenta_zag = cross_model.generate(
            input_ids=x_lenta_zag['input_ids'].to(device).unsqueeze(0),
            attention_mask=x_lenta_zag['attention_mask'].to(device).unsqueeze(0),
            decoder_start_token_id=cross_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )
        
        ria_gen_title = [tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in gen_ids_ria_zag][0]
        lenta_gen_title = [tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in gen_ids_lenta_zag][0]
        
        if i % 100 == 0 and i > 0:
            print(' [SEP] '.join([lenta_gen_title, ria_gen_title]))
            print(p0_vals.count(0) / i * 2)
            print(p1_vals.count(1) / i * 2)
            
        inputs_0 = tokenizer(
            ' [SEP] '.join([lenta_gen_title, ria_gen_title]),
            add_special_tokens=True,
            max_length=250,
            padding="max_length",
            truncation=True
        )
        
        inputs_1 = tokenizer(
            ' [SEP] '.join([ria_gen_title, lenta_gen_title]),
            add_special_tokens=True,
            max_length=250,
            padding="max_length",
            truncation=True
        )
        
        logits_0 = paired_discriminator(input_ids=torch.LongTensor(inputs_0['input_ids']).to(device).unsqueeze(0), 
                               attention_mask=torch.LongTensor(inputs_0['attention_mask']).to(device).unsqueeze(0))[0]
        pred_0 = torch.argmax(logits_0).item()
        
        logits_1 = paired_discriminator(input_ids=torch.LongTensor(inputs_1['input_ids']).to(device).unsqueeze(0), 
                               attention_mask=torch.LongTensor(inputs_1['attention_mask']).to(device).unsqueeze(0))[0]
        pred_1 = torch.argmax(logits_1).item()
        
        p0_vals.append(pred_0)
        p1_vals.append(pred_1)
        

  3%|▎         | 51/2000 [00:39<28:02,  1.16it/s] 

при пожаре в жилом доме на юге москвы погибли два человека [SEP] два человека погибли при пожаре на кухне в жилом доме на юге москвы
0.92
0.94


  5%|▌         | 101/2000 [01:13<26:09,  1.21it/s]

школьники в филадельфии заплатят 600 тысяч долларов за прослушку [SEP] школьник в сша заплатит $610 тысяч за слежку за ноутбуками
0.92
0.92


  8%|▊         | 151/2000 [01:46<18:01,  1.71it/s]

...

двухнедельную девочку нашли живой под завалами в турции [SEP] мать и бабушку из турции спасены из-под завалов
0.894
0.91


 28%|██▊       | 551/2000 [05:59<14:36,  1.65it/s]

российский футбольный союз опроверг назначение алаева гендиректором [SEP] генеральным директором рфс назначен александр алаев
0.8927272727272727
0.9072727272727272


 30%|███       | 601/2000 [06:30<15:03,  1.55it/s]

газировку обязали вкус угольной кислотой [SEP] ученые выяснили, как пузырь вокруг газировки не важен
0.8916666666666667
0.9066666666666666


In [ ]:
p0_vals.count(0) / 2000

In [ ]:
p1_vals.count(1) / 2000

### Paired sicr check

In [42]:
with torch.no_grad():
    p0_vals = []
    p1_vals = []
    
    ria_style_headlines = []
    lenta_style_headlines = []

    for i in tqdm.trange(0, len(dataset), 2):        
        ria_gen_title = dataset.get_strings(i)['title']
        lenta_gen_title = dataset.get_strings(i+1)['title']
        
        if i % 50 == 0 and i > 0:
            print(' [SEP] '.join([lenta_gen_title, ria_gen_title]))
            print(p0_vals.count(0) / i)
            print(p1_vals.count(1) / i)
            
        inputs_0 = tokenizer(
            ' [SEP] '.join([lenta_gen_title, ria_gen_title]),
            add_special_tokens=True,
            max_length=250,
            padding="max_length",
            truncation=True
        )
        
        inputs_1 = tokenizer(
            ' [SEP] '.join([ria_gen_title, lenta_gen_title]),
            add_special_tokens=True,
            max_length=250,
            padding="max_length",
            truncation=True
        )
        
        logits_0 = paired_discriminator(input_ids=torch.LongTensor(inputs_0['input_ids']).to(device).unsqueeze(0), 
                               attention_mask=torch.LongTensor(inputs_0['attention_mask']).to(device).unsqueeze(0))[0]
        pred_0 = torch.argmax(logits_0).item()
        
        logits_1 = paired_discriminator(input_ids=torch.LongTensor(inputs_1['input_ids']).to(device).unsqueeze(0), 
                               attention_mask=torch.LongTensor(inputs_1['attention_mask']).to(device).unsqueeze(0))[0]
        pred_1 = torch.argmax(logits_1).item()
        
        p0_vals.append(pred_0)
        p1_vals.append(pred_1)
        

  1%|▏         | 25/2000 [00:09<17:10,  1.92it/s]

шведская принцесса родила дочь [SEP] кронпринцесса швеции родила первенца - девочку
0.5
0.5


  2%|▎         | 50/2000 [00:19<10:57,  2.97it/s]

при пожаре в москве погибли два человека [SEP] два человека погибли при пожаре в жилом доме в москве
0.5
0.5


  4%|▍         | 75/2000 [00:28<09:54,  3.24it/s]

...

 98%|█████████▊| 1950/2000 [13:05<00:16,  3.09it/s]

водолазы приступили к обследованию сухогруза "амурская" [SEP] водолазы начали погружения к сухогрузу "амурская" в поисках экипажа
0.46897435897435896
0.4712820512820513


 99%|█████████▉| 1975/2000 [13:17<00:11,  2.20it/s]

задолженность в сфере жкх достигла 781 миллиарда рублей [SEP] совокупный долг в сфере жкх составляет на 1 ноября более 780 млрд рублей
0.46936708860759496
0.47164556962025317


100%|██████████| 2000/2000 [13:26<00:00,  2.48it/s]


### Cross style check

In [8]:
from custom_datasets import LentaRiaDataset

In [9]:
cross_model = EncoderDecoderModel.from_pretrained(cross_model_path)

In [11]:
discriminator.cuda();

In [12]:
dataset = LentaRiaDataset('/home/aobuhtijarov/datasets/full_lenta_ria.test.jsonl',
                          tokenizer, 250, 48)

In [13]:
cross_model.cuda();

Even index: lenta $\rightarrow$ ria


Odd index: ria $\rightarrow$ lenta

In [14]:
agency_list

['РИА Новости', 'lenta.ru']

In [15]:
device = 'cpu'
device = 'cuda'

In [16]:
with torch.no_grad():
    ria_good = 0  # accuracy of generating ria from lenta
    lenta_good = 0  # accuracy of generating lenta from ria
    
    ria_style_headlines = []
    lenta_style_headlines = []

    for i in tqdm.trange(len(dataset)):
        x = dataset[i]
        gen_ids = cross_model.generate(
            input_ids=x['input_ids'].to(device).unsqueeze(0),
            attention_mask=x['attention_mask'].to(device).unsqueeze(0),
            decoder_start_token_id=cross_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )

        gen_title = [tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) 
                     for x in gen_ids][0]  
        
        if i % 50 in (0, 1):
            print(gen_title)
            
        if i % 2 == 0:
            ria_style_headlines.append(gen_title)
        else:
            lenta_style_headlines.append(gen_title)
        
        inp = tokenizer(gen_title, 
            add_special_tokens=True, max_length=48,
            padding='max_length', truncation=True
        )

        logits = discriminator(input_ids=torch.LongTensor(inp['input_ids']).to(device).unsqueeze(0), 
                               attention_mask=torch.LongTensor(inp['attention_mask']).to(device).unsqueeze(0))[0]
        pred = torch.argmax(logits).item()

        if i % 2 == 0:
            if pred == agency_to_discr_target['РИА Новости']:
                ria_good += 1
        else:
            if pred == agency_to_discr_target['lenta.ru']:
                lenta_good += 1

  0%|          | 1/4000 [00:04<4:32:23,  4.09s/it]

медведев поздравил коморовский с победой на выборах в польше президент рф президент


  0%|          | 2/4000 [00:04<2:24:29,  2.17s/it]

медведев поздравил коморовский с победой на выборах в польше в кремле медвед


  1%|▏         | 51/4000 [00:30<27:07,  2.43it/s] 

...

 99%|█████████▉| 3951/4000 [30:31<00:33,  1.45it/s]

задолженность населения в сфере жкх в рф превышает 1,5 млрд руб в 2011 г


 99%|█████████▉| 3952/4000 [30:32<00:32,  1.50it/s]

задолженность населения в сфере жкх превысила миллиард рублей в ноябре в минрегионе в


100%|██████████| 4000/4000 [31:01<00:00,  2.15it/s]


In [17]:
cross_ria_acc = ria_good / (len(dataset) / 2)
cross_lenta_acc = lenta_good / (len(dataset) / 2)

In [18]:
cross_ria_acc, cross_lenta_acc

(0.9515, 0.3975)

## Explaining over generated headlines

In [19]:
from transformers_interpret import SequenceClassificationExplainer

In [20]:
agency_list = ["РИА Новости", "lenta.ru"]

agency_to_discr_target = {a: i for i, a in enumerate(sorted(agency_list))}
discr_target_to_agency = {v: k for k, v in agency_to_discr_target.items()}

cname_to_agency = { 
    f'LABEL_{i}': discr_target_to_agency[i] for i in range(len(agency_list))
}

In [21]:
def extract_top_n_words(res, percentile=85, n=20):
    p = np.percentile([x[1] for x in res], percentile)
    top_tokens = [x[0] for x in res if x[1] >= p]
    return Counter(top_tokens).most_common(n)

In [22]:
cls_explainer = SequenceClassificationExplainer(discriminator, tokenizer)

In [23]:
lenta_results = []
lenta_pred_cnt = 0

for r in tqdm.tqdm(lenta_style_headlines, total=len(lenta_style_headlines)):
    val = cls_explainer(r.lower())
    
    if cname_to_agency[cls_explainer.predicted_class_name] == 'lenta.ru':
        lenta_pred_cnt += 1
        lenta_results.extend(val)


 30%|██▉       | 592/2000 [03:09<07:29,  3.13it/s]


KeyboardInterrupt: 

In [ ]:
print(f'Lenta accuracy: {lenta_pred_cnt / len(lenta_style_headlines):.2f}')

In [ ]:
ria_results = []
ria_pred_cnt = 0

for r in tqdm.tqdm(ria_style_headlines, total=len(ria_style_headlines)):
    val = cls_explainer(r.lower())
    if cname_to_agency[cls_explainer.predicted_class_name] == 'РИА Новости':
        ria_pred_cnt += 1
        ria_results.extend(val)


In [ ]:
print('RIA accuracy:', round(ria_pred_cnt / len(ria_style_headlines), 2))

In [ ]:
lenta_results = [(a, b) for a, b in lenta_results if a not in ['[CLS]', '[SEP]']]
ria_results = [(a, b) for a, b in ria_results if a not in ['[CLS]', '[SEP]']]

In [ ]:
from collections import Counter

In [ ]:
extract_top_n_words(lenta_results)

In [ ]:
extract_top_n_words([(x[0], -x[1]) for x in lenta_results])

In [ ]:
extract_top_n_words(ria_results)

In [ ]:
extract_top_n_words([(x[0], -x[1]) for x in ria_results])

## BACKLOG

In [13]:
df = pd.DataFrame(columns=['Data'] + agency_list)

for i, dataset_name in enumerate(('RIA', 'Lenta', 'Other')):
    row = {'Data': dataset_name}
    for j, a in enumerate(agency_list):
        row[a] = result[i][j]      
    df = df.append(row, ignore_index=True)
df

,Data,РИА Новости,lenta.ru
0,RIA,0.935,0.054
1,Lenta,0.426,0.609
2,Other,0.808,0.207


In [14]:
df.index = df.Data

In [15]:
df.drop('Data', axis=1, inplace=True)

In [16]:
df.round(2)

,РИА Новости,lenta.ru
Data,,
RIA,0.94,0.05
Lenta,0.43,0.61
Other,0.81,0.21
